In [ ]:
import os
import json

directory = "/Users/jamierollison/Downloads/Spotify Extended Streaming History"
output_file = "2023_data.json"

# Get all the JSON files in the directory
json_files = [file for file in os.listdir(directory) if file.endswith(".json") and "2023" in file]
print(json_files)

# Initialize an empty list to store the data
data = []

# Iterate over each JSON file
for file in json_files:
    file_path = os.path.join(directory, file)
    with open(file_path, "r") as f:
        # Load the JSON data from the file
        file_data = json.load(f)
        # Append the data to the list
        data.extend([file for file in file_data if '2023' in file['ts']])

# Write the concatenated data to the output file
with open(output_file, "w") as f:
    json.dump(data, f)


In [ ]:
import json
CLIENT_ID="61b7d1ee5dc9443ca8549602fdaa69f0"
CLIENT_SECRET="2958e9c2fbe14b8a8ee17ba8bf42ed6a"

# Specify the file path
file_path = "2023_data.json"

# Read the JSON file
with open(file_path, "r") as f:
    data = json.load(f)

# Create a list of URIs from each entry
uris = [entry["spotify_track_uri"] for entry in data if entry["spotify_track_uri"] != None]

# Print the list of URIs
print(len(uris))

In [ ]:
!pip3 install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


# Chunk the URIs into groups of 100
c = 50
chunked_uris = [uris[i:i+c] for i in range(0, len(uris), c)]

id_map = {}

# Iterate over each chunk of URIs
for chunk in chunked_uris:
    # Query the details of each URI
    results = sp.tracks(chunk)

    # Process the results
    id_map.update({track['id']: {'name': track['name'], 'artists': ", ".join([artist['name'] for artist in track['artists']]), "image": track['album']['images'][0]} for track in results['tracks']})

   


In [ ]:
# from collections import Counter
# c = Counter([uri.split(':')[-1] for uri in uris])
# for song in id_map:
#   del id_map[song]['count']

In [ ]:
months = [{} for _ in range(12)]
for uri in uris:
  month = 

In [ ]:
unique = list(set(uris))

In [ ]:
c = 50
chunked_uris = [unique[i : i + c] for i in range(0, len(unique), c)]

features_cache = {}
for chunk in chunked_uris:
    # Query the details of each URI
    results = sp.audio_features(chunk)
    for track in results:
        try:
            features_cache.update({track['id']: {"valence": track['valence'], "danceability": track['danceability'], "energy": track['energy']}})
        except:
            print(track)

In [78]:
sp.audio_features("2MxAbWhobm2k25T5yDqbah")

[None]

In [ ]:
import datetime

# Define a function to extract the month from a URI
def get_month(entry):
    # Convert the timestamp to a datetime object
    date = datetime.datetime.fromisoformat(entry['ts'])
    # Extract the month from the datetime object
    month = date.month
    return month

# Sort the URIs by the month
songs_by_month = [[] for _ in range(12)]

for entry in data:
    songs_by_month[get_month(entry)-1].append(entry)



In [80]:
from collections import Counter
all_songs = [{} for _ in range(12)]
for i, month_data in enumerate(songs_by_month):
    m_uris = [
        entry["spotify_track_uri"].split(":")[-1]
        for entry in month_data
        if entry["spotify_track_uri"]
        not in [None, "spotify:track:2MxAbWhobm2k25T5yDqbah"]
    ]
    count = Counter(m_uris)

    all_songs[i].update({uri: features_cache[uri] | {"count": count[uri]} for uri in count})
all_songs

[{'4l5PZhHAxCnq5EAOdN0Clh': {'valence': 0.65,
   'danceability': 0.599,
   'energy': 0.618,
   'count': 8},
  '3e9lgSb5IkChkz4higTUOw': {'valence': 0.372,
   'danceability': 0.935,
   'energy': 0.457,
   'count': 5},
  '7CUkeiG7QtB7tPU9f8SANS': {'valence': 0.788,
   'danceability': 0.659,
   'energy': 0.473,
   'count': 14},
  '30aIZLnWYgueY2m3tsMVMz': {'valence': 0.655,
   'danceability': 0.677,
   'energy': 0.62,
   'count': 8},
  '0hfjiQ13PQ4TcHOXyi1NWx': {'valence': 0.82,
   'danceability': 0.706,
   'energy': 0.656,
   'count': 11},
  '3WXhshrs1fzwF3rQE399Gq': {'valence': 0.456,
   'danceability': 0.578,
   'energy': 0.947,
   'count': 13},
  '0TNFCRzOnlgu7W1K47sFnY': {'valence': 0.794,
   'danceability': 0.802,
   'energy': 0.736,
   'count': 3},
  '13DNuiQyuTDcAw6Y8thZFM': {'valence': 0.504,
   'danceability': 0.873,
   'energy': 0.627,
   'count': 2},
  '4V6omSphSf3ql0mXRum3Vh': {'valence': 0.74,
   'danceability': 0.567,
   'energy': 0.754,
   'count': 1},
  '64xy8ja7i7F0VPmgL

In [85]:
from numpy import average

In [86]:
averages = [{} for _ in range(12)]
for i, month in enumerate(all_songs):
    valences = [month[song]["valence"] for song in month]
    energy = [month[song]["energy"] for song in month]
    danceability = [month[song]["danceability"] for song in month]
    count = [month[song]["count"] for song in month]

    averages[i] = {
        "valence": average(valences, weights=count),
        "energy": average(energy, weights=count),
        "danceability": average(danceability, weights=count),
    }

averages

[{'valence': 0.5383968169761273,
  'energy': 0.6771884173297966,
  'danceability': 0.5971663129973475},
 {'valence': 0.47513610108303245,
  'energy': 0.6615683754512635,
  'danceability': 0.5287045487364621},
 {'valence': 0.5350058898847632,
  'energy': 0.7411459667093471,
  'danceability': 0.5629335467349552},
 {'valence': 0.5180233418367347,
  'energy': 0.7279066326530611,
  'danceability': 0.553682206632653},
 {'valence': 0.5452866181410974,
  'energy': 0.6936034714445689,
  'danceability': 0.592247536394177},
 {'valence': 0.5643887145443572,
  'energy': 0.7013940555220277,
  'danceability': 0.632689649969825},
 {'valence': 0.5604544841537423,
  'energy': 0.7186557248819959,
  'danceability': 0.5553414699932568},
 {'valence': 0.550583014084507,
  'energy': 0.6852460845070423,
  'danceability': 0.6091317183098592},
 {'valence': 0.5623939777522056,
  'energy': 0.6947874568469505,
  'danceability': 0.5924762562332182},
 {'valence': 0.5263286783976039,
  'energy': 0.6875131785847998,
  

In [ ]:
with open("jamie_allsongs.js", "w") as f:
    json.dump(all_songs)

with open("jamie_ids_map.js", "w") as f:
    json.dump(id_map)

with open("jamie_averages.js", "w") as f:
    json.dump(averages)